<a href="https://colab.research.google.com/github/Ken-Lau-Lab/single-cell-lectures/blob/main/notebooks2024/Spatial_analysis_section_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## uncomment this part
# ! pip install scanpy
# ! pip install milwrm
# ! pip install squidpy
# ! pip install harmonypy

In [ ]:
import scanpy as sc
import squidpy as sq
import MILWRM as mw
import matplotlib.pyplot as plt

In [ ]:
import logging
logging.getLogger('matplotlib.font_manager').disabled = True

let us look at the lymph node dataset first. Do you recall how can you load this dataset from last class?

In [ ]:
adata = "add your code here" 

Can you do a spatial plot of this data now? 

In [ ]:
"sc.pl...add your code here"

We are going to perform the same normalization steps and perform the PCA on the data that we did in last class. Can you perform the preprocessing steps?

In [4]:
"Add your code here"

'Add your code here'

## MILWRM 
We first create a tissue labeler object defined as variable tl here with the list adata and then we perform preprocessing of this data. Finally we label tissue regions and also estimate confidence score for tissue domain assignment and also look at domain profile for each tissue domain. 

In [ ]:
adatas = [adata] # Create a list of your samples

In [ ]:
tl = mw.st_labeler(adatas)  ## creating a tissue labeler object

While preparing cluster data you can add/reduce features or modify the number of rings. You can read more here on documentation of this function https://ken-lau-lab.github.io/MILWRM/

In [ ]:
%time tl.prep_cluster_data(use_rep="X_pca", features=[0,1,2,3,4,5,6,7,8,9,10,11,12],n_rings=1)

Finally we are labelling tissue domains here. Pay attention to alpha might be useful for an assignment 🤷. Alpha ranges from 0.01 tom 0.05. You will get lesser number of tissue domains the higher your alpha is. Thus, if you want to get tissue domains that are more specific you need to reduce alpha. 

In [ ]:
%time a = tl.label_tissue_regions(plot_out=True, alpha=0.05)

In [ ]:
sc.pl.spatial(adatas[0], color = 'tissue_ID')

In [ ]:
tl.plot_gene_loadings(adata.varm['PCs'], nfiolcols = 3)

tissue ID 0 : CXCL13, FDCSP - stromal network including follicular Dendritic cells
tissue ID 2 : Follicles IGHG2 - Expressed by b cells so follicles
tissue ID 4 : TCF7 - expressed by T cells -- T cell zone
tissue ID 5 : MEF2B - expressed in germinal centers in lymph node

Notice that the genes expressed in respective tissue domains all are associated with a specific function and structure in lymph node. 

### Squidpy Neighborhood analysis

Squidpy library can perform various spatial analysis on a number of different spatial data types. You can read more about its documentation here : https://squidpy.readthedocs.io/en/stable/notebooks/tutorials/tutorial_visium_hne.html
Here we are performing a simple neighborhood enrichment and co-occurence analysis. 
Neighborhood enrichment analysis asses the spatial proximity of clusters, if two spots belonging two different clusters are often close to each other then they are _enriched_.

In [ ]:
sq.gr.spatial_neighbors(adata) ## spatial neighbors functions is use to compute a connectivity matrix for each spot
sq.gr.nhood_enrichment(adata, cluster_key="tissue_ID")
sq.pl.nhood_enrichment(adata, cluster_key="tissue_ID")

co-occurence analysis is similar analysis but it is performed on the actual spot locations and not on the connectivity matrix. 

In [ ]:
sq.gr.co_occurrence(adata, cluster_key="tissue_ID")
sq.pl.co_occurrence(
    adata,
    cluster_key="tissue_ID",
    clusters=2,
    figsize=(8, 4),
)

## MILWRM on mouse brain dataset 

In [ ]:
! curl -O -J -L https://www.dropbox.com/sh/87aqjwn8c0n618w/AADF1ulr-Zy3W3iyMJ9C3kDAa?dl=0

In [ ]:
!unzip mouse_st_data.zip

In [ ]:
mouse_coronal = sc.read("Mouse_coronal/unfiltered_Mouse_coronal.h5ad") ## read in the data mouse_coronal.var_names_make_unique() ## make var names unique

In [ ]:
mouse_saggital_anterior = sc.read("V1_Mouse_brain_saggital/unfiltered_V1_Mouse_brain_saggital_ant.h5ad") mouse_saggital_anterior.var_names_make_unique()

In [ ]:
mouse_saggital_posterior = sc.read("V1_Mouse_brain_saggital_post/unfiltered_V1_Mouse_brain_saggital_post.h5ad") mouse_saggital_posterior.var_names_make_unique()

In [ ]:
adatas = [mouse_coronal, mouse_saggital_anterior, mouse_saggital_posterior]

We need to filter out the spots that are not under the tissue from our data. This is required to make figures with MILWRM. Alternatively you can use filtered data directly for your analysis but you cannot use MILWRM's plotting functions in that case. 

In [ ]:
adata_map = [] 
for adata in adatas: 
    adata_map.append(mw.map_pixels(adata))

In [ ]:
labels = ['coronal', 'sagittal anterior', 'sagittal posterior'] comb = adata_map[0].concatenate(adata_map[1:],fill_value=0, join="outer", batch_categories = labels)

In [ ]:
sc.pp.normalize_total(comb) # normalizing wth respect to the total count so that every cell has same total count at the end. Read more here ->https://scanpy.readthedocs.io/en/stable/generated/scanpy.pp.normalize_total.html 
sc.pp.log1p(comb) # log1p transformation on data 
sc.pp.scale(comb) # scaling the data 
sc.pp.pca(comb, n_comps=50) # performing PCA

In [ ]:
sc.pl.pca_overview(comb, color="batch") # plotting the overview

## Harmonizing the data
Since we perform MILWRM across samples it is important to correct for any batch effects in the data. One way to do it is using Harmonypy a package designed for batch correction
in single cell data. It does batch correction using Principal components. 
You can read more about this package here -> https://www.nature.com/articles/s41592-019-0619-0

In [ ]:
top_20_pcs = comb.obsm['X_pca'][:,:20]  # we are taking 20 PCs here for batch correction

In [ ]:
meta_data = comb.obs 
data_mat = top_20_pcs
data_mat = np.array(data_mat)
vars_use = ['batch']

In [ ]:
ho = hm.run_harmony(data_mat, meta_data, vars_use, max_iter_harmony=10) ## running harmony

In [ ]:
res = pd.DataFrame(ho.Z_corr) 
res.columns = ['X{}'.format(i + 1) for i in range(res.shape[1])]

In [ ]:
res = res.T  ## these are corrected PCs

In [ ]:
comb.obsm['new_PCA'] = res.values  # we will assign those to new PCs in our anndata and use that for MILWRM

In [ ]:
for i,adata in zip(labels,adata_map): ## we are also adding that to our individual anndatas for each sample
    adata.obsm['new_PCA'] = comb[comb.obs.batch==str(i),:].obsm['new_PCA'] 

In [ ]:
for i,adata in zip(labels,adata_map):
    adata.obsm['X_pca'] = comb[comb.obs.batch==str(i),:].obsm['X_pca'] 

## Running MILWRM

We first create a tissue labeler object defined as variable tl here with the list adata and then we perform preprocessing of this data. Finally we label tissue regions and also estimate confidence score for tissue domain assignment and also look at domain profile for each tissue domain. 

In [ ]:
tl = mw.st_labeler(adata_map)  ## Creating tissue labeller object

While preparing cluster data you can add/reduce features or modify the number of rings. You can read more here on documentation of this function https://ken-lau-lab.github.io/MILWRM/


In [ ]:
%time tl.prep_cluster_data(use_rep="new_PCA", features=[0,1,2,3,4,5,6,7,8,9,10,11,12],n_rings=1, n_jobs = 3) 

Finally we are labelling tissue domains here. Pay attention to alpha might be useful for an assignment 🤷. Alpha ranges from 0.01 tom 0.05. You will get lesser number of tissue domains the higher your alpha is. Thus, if you want to get tissue domains that are more specific you need to reduce alpha. 

In [ ]:
%time a = tl.label_tissue_regions(plot_out=True, alpha=0.05, n_jobs = 3)  

#### Let's look at our tissue domains now

In [ ]:
## Looking at Tissue domains
for i in range(len(tl.adatas)):
    # get blurred features that are saved to adata.obs
    # create plot of blurred training features and final tissue_ID labels
    p = mw.assemble_pita(tl.adatas[i], use_rep = ".obs",features=['tissue_ID'], histo = 'hires',cmap = 'tab20', save_to = None)

#### what is proportion of each tissue domain


In [ ]:
a = tl.plot_tissue_ID_proportions_st(save_to=None)

#### confidence scores
How confident are we about our assignments we can do so by plotting the confidence score for spot assignment 

In [ ]:
tl.confidence_score() ## calculating confidence scores
for i in range(len(tl.adatas)):
    # a = sc.pl.spatial(tl.adatas[i],color = "n_confidence_score" )
    p = mw.assemble_pita(tl.adatas[i], use_rep = ".obs",features=['confidence_score'],cmap = 'rainbow', save_to = None)

#### plotting the domain profile for each Tissue domain 

In [ ]:
tl.plot_gene_loadings(comb.varm['PCs'], ncols = 3, save_to= None)